**1. Import library and dataset**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.info
test.info

**2. 1Understanding data **
* - Visualize the percentage of survived and that of unsurvived.
* - Visualize the relation between people's classes and surviving chance.
* - Visualize the relation between agen and surviving chance. 
* -  Visualize the relation between the piers, from which people got on board and surviving chance. 

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.set(alpha=0.2)

In [ ]:
plt.subplot2grid((2,3),(0,0))             
train.Survived.value_counts().plot(kind='bar')# bar graph 
plt.title(u"survival counts (1=survived)") #name
plt.ylabel(u"numbers")

In [ ]:
plt.subplot2grid((2,3),(0,1))
train.Pclass.value_counts().plot(kind="bar")
plt.ylabel(u"numbers")
plt.title(u"cabin classes")

In [ ]:
plt.subplot2grid((2,3),(0,2))
plt.scatter(train.Survived, train.Age)
plt.ylabel(u"age")                         
plt.grid(b=True, which='major', axis='y') 
plt.title(u"survival distribution (1=survived)")


In [ ]:
plt.subplot2grid((2,3),(1,0), colspan=2)
train.Age[train.Pclass == 1].plot(kind='kde')   
train.Age[train.Pclass == 2].plot(kind='kde')
train.Age[train.Pclass == 3].plot(kind='kde')
plt.xlabel(u"age")# plots an axis lable
plt.ylabel(u"density") 
plt.title(u"age distribution of different class")
plt.legend((u'class1', u'class2',u'class3'),loc='best')

In [ ]:
plt.subplot2grid((2,3),(1,2))
train.Embarked.value_counts().plot(kind='bar')
plt.title(u"population from differnt piers")
plt.ylabel(u"numbers")  
plt.show()

**2.2 Statistical Analysis on Survivals(based on Classes, Genders)**

In [ ]:
fig = plt.figure()
fig.set(alpha=0.2)  #set color for the graph
Survived_0 = train.Pclass[train.Survived == 0].value_counts()
Survived_1 = train.Pclass[train.Survived == 1].value_counts()
df=pd.DataFrame({u'being rescued':Survived_1, u'not rescued':Survived_0})
df.plot(kind='bar', stacked=True)
plt.title(u"being rescued or not")
plt.xlabel(u"passengers' classes") 
plt.ylabel(u"numbers") 

In [ ]:
fig = plt.figure()
fig.set(alpha=0.2)  # set color for the graph
 
Survived_0 = train.Embarked[train.Survived == 0].value_counts()
Survived_1 = train.Embarked[train.Survived == 1].value_counts()
df=pd.DataFrame({u'survivor':Survived_1, u'vistim':Survived_0})
df.plot(kind='bar', stacked=True)
plt.title(u"Survival rate of passengers from differnt ports")
plt.xlabel(u"ports") 
plt.ylabel(u"numbers") 
 
plt.show()
fig.set(alpha=0.2)  # set color for the graph
Survived_m = train.Survived[train.Sex == 'male'].value_counts()
Survived_f = train.Survived[train.Sex == 'female'].value_counts()
df=pd.DataFrame({u'male':Survived_m, u'female':Survived_f})
df.plot(kind='bar', stacked=True)
plt.title(u"survival analysis according to gender ")
plt.xlabel(u"gender") 
plt.ylabel(u"number")

In [ ]:
fig = plt.figure()
fig.set(alpha=0.2)  
 
Survived_0 = train.Embarked[train.Survived == 0].value_counts()
Survived_1 = train.Embarked[train.Survived == 1].value_counts()
df=pd.DataFrame({u'survivor':Survived_1, u'victims':Survived_0})
df.plot(kind='bar', stacked=True)
plt.title(u"Survival rate of differnt piers")
plt.xlabel(u"piers") 
plt.ylabel(u"numbers") 
 
plt.show()

**3. Feature Engineering** - Prepare data for statistical modeling
- Check missing values
- Impute the missing Fare values with the mean Fare value
- Impute the missing Age values with the mean Age value
- Impute the missing Cabin values with the mean Cabin value
- Extract title from name, Extract new features from the other features

In [ ]:
print((train.Fare == 0).sum())

In [ ]:
import numpy as np
train.Fare = train.Fare.replace(0, np.NaN)
print ((train.Fare == 0).sum())

In [ ]:
train[train.Fare.isnull()].index

In [ ]:
train.Fare.mean()

In [ ]:
train.Fare.fillna(train.Fare.mean(),inplace=True)
train[train.Fare.isnull()]

In [ ]:
print((train.Age == 0).sum())#no need to impute
train.Age.fillna(train.Age.mean(),inplace=True)
train[train.Age.isnull()]

In [ ]:
train.Cabin.isnull().mean() # no need to impute

In [ ]:
train['Name_len']=train.Name.str.len()
train['Ticket_First']=train.Ticket.str[0]
train['FamilyCount']=train.SibSp+train.Parch
train['Cabin_First']=train.Cabin.str[0]

In [ ]:
# Regular expression to get the title of the Name
train['title'] = train.Name.str.extract('\, ([A-Z][^ ]*\.)',expand=False)
train.title.value_counts().reset_index()

**4. Statistical Modeling** -- Choose the most accurate model

In [ ]:
train.columns
trainML = train[['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Embarked', 'Name_len', 'Ticket_First', 'FamilyCount',
       'title']]
trainML = trainML.dropna()
trainML.isnull().sum()

**A single predictor model with logistic regression**
- Regression on survival on Age
- Regression on survival on Fare
- Regression on survival on Sex
- Regression on Survival on Passenger Class
- Predicting Survival based on Random forest model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

* **4.1 regression analysis **On age

In [ ]:
X_Age = trainML[['Age']].values
y = trainML['Survived'].values
# Use the fit method to train
lr.fit(X_Age,y)
# Make a prediction
y_predict = lr.predict(X_Age)
y_predict[:10]
(y == y_predict).mean()
#The prediction accuracy is marginally better than the base line accuracy of 61.5% which we got earlier

**4.2 regression analysis**On Fare

In [ ]:
X_Fare = trainML[['Fare']].values
y = trainML['Survived'].values
# Use the fit method to train
lr.fit(X_Fare,y)
# Make a prediction
y_predict = lr.predict(X_Fare)
y_predict[:10]
(y == y_predict).mean()

***4.3 regression analysis***On Gender

In [ ]:
X_sex = pd.get_dummies(trainML['Sex']).values
y = trainML['Survived'].values
# Use the fit method to train
lr.fit(X_sex, y)
# Make a prediction
y_predict = lr.predict(X_sex)
y_predict[:10]
(y == y_predict).mean()

**4.4 regression analysis **On Passengers'Classes

In [ ]:
X_pclass = pd.get_dummies(trainML['Pclass']).values
y = trainML['Survived'].values
lr = LogisticRegression()
lr.fit(X_pclass, y)
# Make a prediction
y_predict = lr.predict(X_pclass)
y_predict[:10]
(y == y_predict).mean()

**4.5 Predicting the survival using Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X=trainML[['Age', 'SibSp', 'Parch',
       'Fare', 'Name_len', 'FamilyCount']].values # Taking all the numerical values
y = trainML['Survived'].values
RF = RandomForestClassifier()
RF.fit(X, y)
# Make a prediction
y_predict = RF.predict(X)
y_predict[:10]
(y == y_predict).mean()

**5. Examine test data as well**

In [ ]:
test_dataset = pd.read_csv('../input/test.csv')



In [ ]:
def data_process(data):
    data["Fare"] = data["Fare"].fillna(data["Fare"].dropna().median())
    data["Age"] = data["Age"].fillna(data["Age"].dropna().median())
    
    
    
    data = data.drop(['Fare'], axis=1)
    data = data.drop(['Ticket'], axis=1)
    data = data.drop(['Cabin'], axis=1)
    freq_port = test_dataset.Embarked.dropna().mode()[0]


    data['Embarked'] = data['Embarked'].fillna(freq_port)

    data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    
    data = data.drop(['Name'], axis=1)
    
    data.loc[data["Sex"] == "male", "Sex"] = 0
    data.loc[data["Sex"] == "female", "Sex"] = 1
      
    data.loc[ data['Age'] <= 16, 'Age'] = int(0)
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[data['Age'] > 64, 'Age']
    
    return data

**7. Predict on test.dataset using Random Forest Model & Submit** 

In [ ]:
import utils
X_test_dataset = data_process(test_dataset)
X_test_dataset = X_test_dataset.drop("PassengerId", axis=1)
X_test_dataset.head()
predicted_value = RF.predict(X_test_dataset)

In [ ]:
test_dataset_copy = pd.read_csv('../input/test.csv')
submission = pd.DataFrame({
        "PassengerId": test_dataset_copy["PassengerId"],
        "Survived": predicted_value
})

submission.to_csv('submission.csv', index=False)